In [ ]:
# Use the Elhub API to retrieve hourly production data for all price areas using PRODUCTION_PER_GROUP_MBA_HOUR for all days and hours of the year 2021.
# The API allows retrieving different datasets for each entity using the following format: https://api.elhub.no/energy-data/v0/{entity}?dataset={dataset}
# The API returns data in JSON format. You can use the requests library to make the API calls and the pandas library to process the data.
# You can use the following code to get started:
import requests
import pandas as pd
import json
import datetime as dt
import matplotlib.pyplot as plt
# Define the API endpoint and parameters
endpoint = "https://api.elhub.no/energy-data/v0/price-areas"
dataset = "PRODUCTION_PER_GROUP_MBA_HOUR"
start_date = "2021-01-01T00:00:00Z"
end_date = "2021-12-31T23:00:00Z"
# Make the API call
url = f"{endpoint}?dataset={dataset}&start={start_date}&end={end_date}"
response = requests.get(url)
print(f"Status code: {response.status_code}")
data = response.json()



Status code: 200
{'data': [{'attributes': {'country': 'NO', 'eic': '*', 'name': '*', 'productionPerGroupMbaHour': []}, 'id': '*', 'type': 'price-areas'}, {'attributes': {'country': 'NO', 'eic': '10YNO-1--------2', 'name': 'NO1', 'productionPerGroupMbaHour': [{'endTime': '2025-09-16T19:00:00+02:00', 'lastUpdatedTime': '2025-09-30T18:07:59+02:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 2644987.0, 'startTime': '2025-09-16T18:00:00+02:00'}, {'endTime': '2025-09-16T20:00:00+02:00', 'lastUpdatedTime': '2025-09-30T18:07:59+02:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 2599483.2, 'startTime': '2025-09-16T19:00:00+02:00'}, {'endTime': '2025-09-16T21:00:00+02:00', 'lastUpdatedTime': '2025-09-30T18:07:59+02:00', 'priceArea': 'NO1', 'productionGroup': 'hydro', 'quantityKwh': 2577709.8, 'startTime': '2025-09-16T20:00:00+02:00'}, {'endTime': '2025-09-16T22:00:00+02:00', 'lastUpdatedTime': '2025-09-30T18:07:59+02:00', 'priceArea': 'NO1', 'productionGro

In [5]:
data = response.json()

# Print top-level keys
print("Top-level keys:", list(data.keys()))

# Print first element of 'data' if it exists
if "data" in data:
    print("First entry keys:", data["data"][0].keys())
    print("Attributes keys:", data["data"][0]["attributes"].keys())
else:
    print("No 'data' key in response")


Top-level keys: ['data', 'links', 'meta']
First entry keys: dict_keys(['attributes', 'id', 'type'])
Attributes keys: dict_keys(['country', 'eic', 'name', 'productionPerGroupMbaHour'])


In [6]:
all_records = []

for entry in data.get("data", []):
    attrs = entry.get("attributes", {})
    recs = attrs.get("productionPerGroupMbaHour", [])
    # Filter out placeholder "*"
    recs = [r for r in recs if r.get("productionGroup") != "*"]
    all_records.extend(recs)

print(f"Total records collected: {len(all_records)}")


Total records collected: 17550


In [9]:
df = pd.DataFrame(all_records)

# Convert timestamps and numeric columns
df['startTime'] = pd.to_datetime(df['startTime'])
df['quantityKwh'] = pd.to_numeric(df['quantityKwh'])

# Optional: set startTime as index
df.set_index('startTime', inplace=True)

df.head()


,endTime,lastUpdatedTime,priceArea,productionGroup,quantityKwh
startTime,,,,,
2025-09-16 18:00:00+02:00,2025-09-16T19:00:00+02:00,2025-09-30T18:07:59+02:00,NO1,hydro,2644987.0
2025-09-16 19:00:00+02:00,2025-09-16T20:00:00+02:00,2025-09-30T18:07:59+02:00,NO1,hydro,2599483.2
2025-09-16 20:00:00+02:00,2025-09-16T21:00:00+02:00,2025-09-30T18:07:59+02:00,NO1,hydro,2577709.8
2025-09-16 21:00:00+02:00,2025-09-16T22:00:00+02:00,2025-09-30T18:07:59+02:00,NO1,hydro,2554189.2
2025-09-16 22:00:00+02:00,2025-09-16T23:00:00+02:00,2025-09-30T18:07:59+02:00,NO1,hydro,2513586.0


In [10]:
print(df['priceArea'].unique())
print(df['productionGroup'].unique())
print(df.index.min(), df.index.max())


['NO1' 'NO2' 'NO3' 'NO4' 'NO5']
['hydro' 'other' 'solar' 'thermal' 'wind']
2025-09-16 18:00:00+02:00 2025-10-15 23:00:00+02:00
